In [1]:
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
import re
import time
import collections
import os

In [2]:
def build_dataset(words, n_words, atleast=1):
    count = [['PAD', 0], ['GO', 1], ['EOS', 2], ['UNK', 3]]
    counter = collections.Counter(words).most_common(n_words)
    counter = [i for i in counter if i[1] >= atleast]
    count.extend(counter)
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [3]:
lines = open('movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conv_lines = open('movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]
        
convs = [ ]
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    convs.append(_line.split(','))
    
questions = []
answers = []

for conv in convs:
    for i in range(len(conv)-1):
        questions.append(id2line[conv[i]])
        answers.append(id2line[conv[i+1]])
        
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    return ' '.join([i.strip() for i in filter(None, text.split())])

clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
    
clean_answers = []    
for answer in answers:
    clean_answers.append(clean_text(answer))
    
min_line_length = 2
max_line_length = 5
short_questions_temp = []
short_answers_temp = []

i = 0
for question in clean_questions:
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length:
        short_questions_temp.append(question)
        short_answers_temp.append(clean_answers[i])
    i += 1

short_questions = []
short_answers = []

i = 0
for answer in short_answers_temp:
    if len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
        short_answers.append(answer)
        short_questions.append(short_questions_temp[i])
    i += 1
    
question_test = short_questions[500:550]
answer_test = short_answers[500:550]
short_questions = short_questions[:500]
short_answers = short_answers[:500]

In [4]:
concat_from = ' '.join(short_questions+question_test).split()
vocabulary_size_from = len(list(set(concat_from)))
data_from, count_from, dictionary_from, rev_dictionary_from = build_dataset(concat_from, vocabulary_size_from)
print('vocab from size: %d'%(vocabulary_size_from))
print('Most common words', count_from[4:10])
print('Sample data', data_from[:10], [rev_dictionary_from[i] for i in data_from[:10]])
print('filtered vocab size:',len(dictionary_from))
print("% of vocab used: {}%".format(round(len(dictionary_from)/vocabulary_size_from,4)*100))

vocab from size: 657
Most common words [('you', 132), ('is', 78), ('i', 68), ('what', 51), ('it', 50), ('that', 49)]
Sample data [7, 28, 129, 35, 61, 42, 12, 22, 82, 225] ['what', 'good', 'stuff', 'she', 'okay', 'they', 'do', 'to', 'hey', 'sweet']
filtered vocab size: 661
% of vocab used: 100.61%


In [5]:
concat_to = ' '.join(short_answers+answer_test).split()
vocabulary_size_to = len(list(set(concat_to)))
data_to, count_to, dictionary_to, rev_dictionary_to = build_dataset(concat_to, vocabulary_size_to)
print('vocab from size: %d'%(vocabulary_size_to))
print('Most common words', count_to[4:10])
print('Sample data', data_to[:10], [rev_dictionary_to[i] for i in data_to[:10]])
print('filtered vocab size:',len(dictionary_to))
print("% of vocab used: {}%".format(round(len(dictionary_to)/vocabulary_size_to,4)*100))

vocab from size: 660
Most common words [('i', 97), ('you', 91), ('is', 62), ('it', 58), ('not', 47), ('what', 39)]
Sample data [12, 216, 5, 4, 94, 25, 59, 10, 8, 79] ['the', 'real', 'you', 'i', 'hope', 'so', 'they', 'do', 'not', 'hi']
filtered vocab size: 664
% of vocab used: 100.61%


In [6]:
GO = dictionary_from['GO']
PAD = dictionary_from['PAD']
EOS = dictionary_from['EOS']
UNK = dictionary_from['UNK']

In [7]:
for i in range(len(short_answers)):
    short_answers[i] += ' EOS'

In [8]:
def str_idx(corpus, dic):
    X = []
    for i in corpus:
        ints = []
        for k in i.split():
            ints.append(dic.get(k,UNK))
        X.append(ints)
    return X

def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

In [9]:
X = str_idx(short_questions, dictionary_from)
Y = str_idx(short_answers, dictionary_to)
X_test = str_idx(question_test, dictionary_from)
Y_test = str_idx(answer_test, dictionary_from)

In [10]:
emb_size = 256
n_hidden = 256
n_layers = 4
n_attn_heads = 16
learning_rate = 1e-4
batch_size = 16
epoch = 20

In [11]:
def encoder_block(inp, n_hidden, filter_size):
    inp = tf.expand_dims(inp, 2)
    inp = tf.pad(inp, [[0, 0], [(filter_size[0]-1)//2, (filter_size[0]-1)//2], [0, 0], [0, 0]])
    conv = tf.layers.conv2d(inp, n_hidden, filter_size, padding="VALID", activation=None)
    conv = tf.squeeze(conv, 2)
    return conv

def decoder_block(inp, n_hidden, filter_size):
    inp = tf.expand_dims(inp, 2)
    inp = tf.pad(inp, [[0, 0], [filter_size[0]-1, 0], [0, 0], [0, 0]])
    conv = tf.layers.conv2d(inp, n_hidden, filter_size, padding="VALID", activation=None)
    conv = tf.squeeze(conv, 2)
    return conv

def glu(x):
    return tf.multiply(x[:, :, :tf.shape(x)[2]//2], tf.sigmoid(x[:, :, tf.shape(x)[2]//2:]))

def layer(inp, conv_block, kernel_width, n_hidden, residual=None):
    z = conv_block(inp, n_hidden, (kernel_width, 1))
    return glu(z) + (residual if residual is not None else 0)

In [12]:
class Chatbot:
    def __init__(self):

        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])

        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype = tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype = tf.int32)
        batch_size = tf.shape(self.X)[0]
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        
        encoder_embedding = tf.Variable(tf.random_uniform([len(dictionary_from), emb_size], -1, 1))
        decoder_embedding = tf.Variable(tf.random_uniform([len(dictionary_to), emb_size], -1, 1))
        
        def forward(x, y,reuse=False):
            with tf.variable_scope('forward',reuse=reuse):
                encoder_embedded = tf.nn.embedding_lookup(encoder_embedding, x)
                decoder_embedded = tf.nn.embedding_lookup(decoder_embedding, y)

                e = tf.identity(encoder_embedded)

                for i in range(n_layers):
                    z = layer(encoder_embedded, encoder_block, 3, n_hidden * 2, encoder_embedded)
                    encoder_embedded = z

                encoder_output, output_memory = z, z + e
                g = tf.identity(decoder_embedded)

                for i in range(n_layers):
                    attn_res = h = layer(decoder_embedded, decoder_block, 3, n_hidden * 2, 
                                         residual=tf.zeros_like(decoder_embedded))
                    C = []
                    for j in range(n_attn_heads):
                        h_ = tf.layers.dense(h, n_hidden//n_attn_heads)
                        g_ = tf.layers.dense(g, n_hidden//n_attn_heads)
                        zu_ = tf.layers.dense(encoder_output, n_hidden//n_attn_heads)
                        ze_ = tf.layers.dense(output_memory, n_hidden//n_attn_heads)

                        d = tf.layers.dense(h_, n_hidden//n_attn_heads) + g_
                        dz = tf.matmul(d, tf.transpose(zu_, [0, 2, 1]))
                        a = tf.nn.softmax(dz)
                        c_ = tf.matmul(a, ze_)
                        C.append(c_)

                    c = tf.concat(C, 2)
                    h = tf.layers.dense(attn_res + c, n_hidden)
                    decoder_embedded = h

                decoder_output = tf.sigmoid(h)
                return tf.layers.dense(decoder_output, len(dictionary_to))
        self.training_logits = forward(self.X, decoder_input)
        self.logits = forward(self.X, self.Y, reuse=True)
        self.k = tf.placeholder(dtype = tf.int32)
        p = tf.nn.softmax(self.logits)
        self.topk_logprobs, self.topk_ids = tf.nn.top_k(tf.log(p), self.k)
        
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [13]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Chatbot()
sess.run(tf.global_variables_initializer())

In [14]:
for i in range(epoch):
    total_loss, total_accuracy = 0, 0
    for k in range(0, len(short_questions), batch_size):
        index = min(k+batch_size, len(short_questions))
        batch_x, seq_x = pad_sentence_batch(X[k: index], PAD)
        batch_y, seq_y = pad_sentence_batch(Y[k: index], PAD)
        accuracy,loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                                      feed_dict={model.X:batch_x,
                                                model.Y:batch_y})
        total_loss += loss
        total_accuracy += accuracy
    total_loss /= (len(short_questions) / batch_size)
    total_accuracy /= (len(short_questions) / batch_size)
    print('epoch: %d, avg loss: %f, avg accuracy: %f'%(i+1, total_loss, total_accuracy))

epoch: 1, avg loss: 6.553852, avg accuracy: 0.033602
epoch: 2, avg loss: 5.719022, avg accuracy: 0.209020
epoch: 3, avg loss: 5.209505, avg accuracy: 0.238536
epoch: 4, avg loss: 4.920930, avg accuracy: 0.249183
epoch: 5, avg loss: 4.714801, avg accuracy: 0.263082
epoch: 6, avg loss: 4.558804, avg accuracy: 0.271176
epoch: 7, avg loss: 4.430582, avg accuracy: 0.284878
epoch: 8, avg loss: 4.323740, avg accuracy: 0.293317
epoch: 9, avg loss: 4.211672, avg accuracy: 0.309034
epoch: 10, avg loss: 4.102667, avg accuracy: 0.324224
epoch: 11, avg loss: 3.994004, avg accuracy: 0.337879
epoch: 12, avg loss: 3.905378, avg accuracy: 0.346359
epoch: 13, avg loss: 3.809965, avg accuracy: 0.359319
epoch: 14, avg loss: 3.710252, avg accuracy: 0.368882
epoch: 15, avg loss: 3.622034, avg accuracy: 0.382101
epoch: 16, avg loss: 3.533204, avg accuracy: 0.395169
epoch: 17, avg loss: 3.446505, avg accuracy: 0.412381
epoch: 18, avg loss: 3.371806, avg accuracy: 0.419066
epoch: 19, avg loss: 3.286955, avg ac

In [15]:
class Hypothesis:
    def __init__(self, log_prob, seq):
        self.log_prob = log_prob
        self.seq = seq

    @property
    def step(self):
        return len(self.seq) - 1


def beam_search(
    batch_x,
    beam_size,
    num_ans = 5,
    normalize_by_len = 1.0,
):
    assert 0 <= normalize_by_len <= 1
    batch_size = len(batch_x)
    max_len = len(batch_x[0]) * 2
    dec_inputs = np.ones((batch_size, 2), dtype=np.int32)
    answers = [[] for i in range(batch_size)]
    H = [[] for i in range(batch_size)]
    
    tkl, tkid = sess.run([model.topk_logprobs, 
                          model.topk_ids],
                         feed_dict = {model.X: batch_x,
                                     model.Y: dec_inputs,
                                     model.k: beam_size})
    for i in range(batch_size):
        for j, log_prob in enumerate(tkl[i, 0]):
            if tkid[i, 0, j] != EOS:
                h = Hypothesis(log_prob, [1, tkid[i, 0, j]])
                H[i].append(h)
        H[i].sort(key=lambda h: h.log_prob)
    
    done = [False] * batch_size
    while not all(done):
        tkl_beam = []
        tkid_beam = []
        dec_inputs_beam = []
        steps_beam = []
        for i in range(beam_size):
            steps = [1] * batch_size
            prev_log_probs = np.zeros(batch_size, dtype=np.float32)
            dec_inputs = np.ones((batch_size, max_len), dtype=np.int32)
            for j, h in enumerate(H):
                while h:
                    hi = h.pop()
                    lp, step, candidate_seq = hi.log_prob, hi.step, hi.seq
                    if candidate_seq[-1] != EOS:
                        dec_inputs[j, :len(candidate_seq)] = candidate_seq
                        steps[j] = step
                        prev_log_probs[j] = lp
                        break
                    else:
                        answers[j].append((lp, candidate_seq))
            max_step = max(steps)
            dec_inputs = dec_inputs[:, :max_step + 2]
            tkl, tkid = sess.run([model.topk_logprobs, 
                          model.topk_ids],
                         feed_dict = {model.X: batch_x,
                                     model.Y: dec_inputs,
                                     model.k: beam_size})
            
            tkl_beam.append(tkl + prev_log_probs[:, None, None])
            tkid_beam.append(tkid)
            dec_inputs_beam.append(dec_inputs.copy())
            steps_beam.append(steps)
            
        for i in range(beam_size):
            tkl = tkl_beam[i]
            tkid = tkid_beam[i]
            dec_inputs = dec_inputs_beam[i]
            steps = steps_beam[i]
            for j in range(batch_size):
                step = steps[j]
                for k in range(tkid.shape[2]):
                    extended_seq = np.hstack((dec_inputs[j, :step+1], [tkid[j, step, k]]))
                    log_prob = tkl[j, step, k]
                    if len(extended_seq) <= max_len and log_prob > -10:
                        h = Hypothesis(log_prob, extended_seq)
                        H[j].append(h)
                H[j].sort(key=lambda h: h.log_prob / (h.step**normalize_by_len))
            
        for i in range(batch_size):
            done[i] = (len(answers[i]) >= num_ans) or (not H[i]) or (len(H[i]) > 100)
            
    return answers    

In [16]:
beamed = beam_search(batch_x, 5)

In [17]:
beamed[0]

[(-4.013502, array([1, 1, 2], dtype=int32)),
 (-3.135856, array([ 1,  1, 11,  2], dtype=int32)),
 (-4.1007824, array([1, 1, 6, 2], dtype=int32)),
 (-4.2032185, array([1, 1, 4, 2], dtype=int32)),
 (-4.2108245, array([  1,  11, 619,   2], dtype=int32)),
 (-4.79195, array([1, 1, 8, 2], dtype=int32)),
 (-6.214799, array([ 1,  1, 11, 11,  2], dtype=int32)),
 (-4.7734714, array([ 1, 11,  5,  2], dtype=int32)),
 (-6.903605, array([ 1,  1, 11,  5,  2], dtype=int32)),
 (-7.131966, array([ 1,  1, 11,  8,  2], dtype=int32)),
 (-7.1793413, array([ 1,  1,  6, 11,  2], dtype=int32)),
 (-7.24696, array([ 1,  1, 11,  7,  2], dtype=int32)),
 (-7.655948, array([1, 1, 8, 5, 2], dtype=int32)),
 (-7.7237387, array([1, 1, 6, 5, 2], dtype=int32)),
 (-7.808584, array([ 1,  1,  8, 11,  2], dtype=int32)),
 (-8.093112, array([ 1,  1,  4, 11,  2], dtype=int32))]

In [18]:
predicted = [max(b, key = lambda t: t[0])[1] for b in beamed]

In [19]:
for i in range(len(batch_x)):
    print('row %d'%(i+1))
    print('QUESTION:',' '.join([rev_dictionary_from[n] for n in batch_x[i] if n not in [0,1,2,3]]))
    print('REAL ANSWER:',' '.join([rev_dictionary_to[n] for n in batch_y[i] if n not in[0,1,2,3]]))
    print('PREDICTED ANSWER:',' '.join([rev_dictionary_to[n] for n in predicted[i] if n not in[0,1,2,3]]),'\n')

row 1
QUESTION: i am a werewolf
REAL ANSWER: a werewolf
PREDICTED ANSWER: a 

row 2
QUESTION: i was dreaming again
REAL ANSWER: i would think so
PREDICTED ANSWER: i 

row 3
QUESTION: the kitchen
REAL ANSWER: very nice
PREDICTED ANSWER: you is 

row 4
QUESTION: the bedroom
REAL ANSWER: there is only one bed
PREDICTED ANSWER: you is 



In [20]:
batch_x, seq_x = pad_sentence_batch(X_test[:batch_size], PAD)
batch_y, seq_y = pad_sentence_batch(Y_test[:batch_size], PAD)
beamed = beam_search(batch_x, 5)
predicted = [max(b, key = lambda t: t[0])[1] for b in beamed]

for i in range(len(batch_x)):
    print('row %d'%(i+1))
    print('QUESTION:',' '.join([rev_dictionary_from[n] for n in batch_x[i] if n not in [0,1,2,3]]))
    print('REAL ANSWER:',' '.join([rev_dictionary_to[n] for n in batch_y[i] if n not in[0,1,2,3]]))
    print('PREDICTED ANSWER:',' '.join([rev_dictionary_to[n] for n in predicted[i] if n not in[0,1,2,3]]),'\n')

row 1
QUESTION: but david
REAL ANSWER: is here that
PREDICTED ANSWER:  

row 2
QUESTION: hopeless it is hopeless
REAL ANSWER: tell ballet then back
PREDICTED ANSWER:  

row 3
QUESTION: miss price
REAL ANSWER: yes learning
PREDICTED ANSWER: you 

row 4
QUESTION: mr kessler wake up please
REAL ANSWER: is here are
PREDICTED ANSWER:  

row 5
QUESTION: there were witnesses
REAL ANSWER: why she out
PREDICTED ANSWER: you he 

row 6
QUESTION: what about it
REAL ANSWER: not you are
PREDICTED ANSWER:  

row 7
QUESTION: go on ask them
REAL ANSWER: i just home
PREDICTED ANSWER:  

row 8
QUESTION: beware the moon
REAL ANSWER: seen hi is he
PREDICTED ANSWER: i 

row 9
QUESTION: did you hear that
REAL ANSWER: is down what
PREDICTED ANSWER:  

row 10
QUESTION: i heard that
REAL ANSWER: it here not
PREDICTED ANSWER: i 

row 11
QUESTION: the hound of the baskervilles
REAL ANSWER: heard
PREDICTED ANSWER: i 

row 12
QUESTION: it is moving
REAL ANSWER: not you hear
PREDICTED ANSWER: you 

row 13
QUESTION: 